# COMP 2020

## 5. gRPC

### 5.1. Tutorial

Creating gRPC application:

In [ ]:
! dotnet new grpc -o Server

Now we can run the application and see some info in the browser at [https://localhost:5001](https://localhost:5001):

In [ ]:
! cd Server && dotnet run

Now stop the above cell.

The message redirects to [Create the gRPC client in a .NET console app](https://docs.microsoft.com/en-us/aspnet/core/tutorials/grpc/grpc-start?view=aspnetcore-3.1&tabs=visual-studio#create-the-grpc-client-in-a-net-console-app) tutorial.

Let us create an empty client application:

In [ ]:
! dotnet new console -o Client

Add required packages:

In [ ]:
! cd Client && dotnet add package Grpc.Net.Client

In [ ]:
! cd Client && dotnet add package Google.Protobuf

In [ ]:
! cd Client && dotnet add package Grpc.Tools

Adding ProtoBuf files:

In [ ]:
! mkdir Client/Protos

In [ ]:
! cp Server/Protos/greet.proto Client/Protos/greet.proto

Now edit the [Client/Protos/greet.proto](../../../edit/05_grpc/01_tutorial/Client/Protos/greet.proto) and update:

```proto
option csharp_namespace = "GrpcGreeterClient";
```

Now edit the [Client/Client.csproj](../../../edit/05_grpc/01_tutorial/Client/Client.csproj) and add:

```xml
<ItemGroup>
  <Protobuf Include="Protos\greet.proto" GrpcServices="Client" />
</ItemGroup>
```

Edit the [Client/Program.cs](../../../edit/05_grpc/01_tutorial/Client/Program.cs) and add:
    
```c#
using System;
using System.Net.Http;
using System.Threading.Tasks;
using Grpc.Net.Client;

namespace GrpcGreeterClient
{
    class Program
    {
        static async Task Main(string[] args)
        {
            using var channel = GrpcChannel.ForAddress("https://localhost:5001");
            var client =  new Greeter.GreeterClient(channel);
            var request = new HelloRequest { Name = "GreeterClient" };
            var reply = await client.SayHelloAsync(request);
            Console.WriteLine("Greeting: " + reply.Message);
        }
    }
}
```

Try building the application:

In [ ]:
! cd Client && dotnet build

Check what kind of error is visible when server is not running:

In [ ]:
! cd Client && dotnet run

Starting the server:

In [ ]:
import subprocess
server = subprocess.Popen(['dotnet', 'run', '--project', 'Server'])

In [ ]:
! cd Client && dotnet run

To fix the certifacate issues edit the [Client/Program.cs](../../../edit/05_grpc/01_tutorial/Client/Program.cs) and replace:

```c#
using var channel = GrpcChannel.ForAddress("https://localhost:5001");
```

by:

```c#
var httpHandler = new HttpClientHandler();

httpHandler.ServerCertificateCustomValidationCallback = 
    HttpClientHandler.DangerousAcceptAnyServerCertificateValidator;

using var channel = GrpcChannel.ForAddress("https://localhost:5001",
    new GrpcChannelOptions { HttpHandler = httpHandler });
```

In [ ]:
! cd Client && dotnet run

In [ ]:
server.kill()

Cleanup:

In [ ]:
! rm -rf Server Client